In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
%matplotlib inline
#%matplotlib notebook
sns.set(rc={'figure.figsize':(11.7,8.27)})
from scipy import stats

In [3]:
#data = pd.read_pickle('df_tesco_marker_V2.pkl')

In [12]:
# calcualting change in exclusive shop sales from median exclusive shop sales for  a site #
data['exclShopSales'] = data['Total Sales Value']-data['Sales Baskets With Fuel']
data['medianExclShopSalesSiteWise'] = data.groupby('PBL')['exclShopSales'].transform('median')
data['pctChgExclShopSalesFromMedian'] = (data['exclShopSales'] / data['medianExclShopSalesSiteWise']-1)*100

In [15]:
# calcualting change in  shop sales from Fuel basket from median shop sales  from Fuels for  a site #
data['shopSalesFromFuel'] = data['Sales Baskets With Fuel']-data['totalSalesEsso']
data['medianShopSalesFromFuelSiteWise'] = data.groupby('PBL')['shopSalesFromFuel'].transform('median')
data['pctChgShopSalesFromFuelFromMedian'] = (data['shopSalesFromFuel'] / data['medianShopSalesFromFuelSiteWise']-1)*100

In [16]:
# calcualting change in  fuel sales  from median Fuels for  a site #
data['medianFuelSalesEssoSiteWise'] = data.groupby('PBL')['totalSalesEsso'].transform('median')
data['pctChgFuelSalesFromMedian'] = (data['totalSalesEsso'] / data['medianFuelSalesEssoSiteWise']-1)*100

In [2]:
data = pd.read_pickle('df_tesco_marker_V3.pkl')

In [30]:
# ealsticity calcualtion for excluive shop sales with respect to ADO price Position 
def regRes(x):
    # filter all data of the site between 5th & 95th precentile 
    q_5,q_95 = x['pctChgExclShopSalesFromMedian'].quantile([0.02,0.98]).values.round().tolist()
    xf = x.query('@q_5 <= pctChgExclShopSalesFromMedian <= @q_95 ')
    slope,intercept,rvalu,pvalue,stderr= stats.linregress(xf['AdoApprPriceChgPct'],\
                                                          xf['pctChgExclShopSalesFromMedian'])
    return(pd.Series([slope,intercept,rvalu,pvalue,stderr],\
                     index=['slope','intercept','rvalu','pvalue','stderr']))

regExclShopSalesVsAppPriceADO_pct = data.groupby('Site Name').apply(regRes).reset_index()

regExclShopSalesVsAppPriceADO_pct_sig = regExclShopSalesVsAppPriceADO_pct.query('pvalue <= 0.05')

regExclShopSalesVsAppPriceADO_pct_sig.shape

# ealsticity calcualtion for shop slaes from Fuel with respect to ADO price Position 
def regRes(x):
    # filter all data of the site between 5th & 95th precentile 
    q_5,q_95 = x['pctChgShopSalesFromFuelFromMedian'].quantile([0.02,0.98]).values.round().tolist()
    xf = x.query('@q_5 <= pctChgShopSalesFromFuelFromMedian <= @q_95 ')
    slope,intercept,rvalu,pvalue,stderr= stats.linregress(xf['AdoApprPriceChgPct'],\
                                                          xf['pctChgShopSalesFromFuelFromMedian'])
    return(pd.Series([slope,intercept,rvalu,pvalue,stderr],\
                     index=['slope','intercept','rvalu','pvalue','stderr']))

regShopSalesFromFuelVsAppPriceADO_pct = data.groupby('Site Name').apply(regRes).reset_index()

regShopSalesFromFuelVsAppPriceADO_pct_sig = regShopSalesFromFuelVsAppPriceADO_pct.query('pvalue <= 0.05')

regShopSalesFromFuelVsAppPriceADO_pct_sig.shape

# ealsticity calcualtion for fuel sales with respect to ADO price Position 
def regRes(x):
    # filter all data of the site between 5th & 95th precentile 
    q_5,q_95 = x['pctChgFuelSalesFromMedian'].quantile([0.02,0.98]).values.round().tolist()
    xf = x.query('@q_5 <= pctChgFuelSalesFromMedian <= @q_95 ')
    slope,intercept,rvalu,pvalue,stderr= stats.linregress(xf['AdoApprPriceChgPct'],\
                                                          xf['pctChgFuelSalesFromMedian'])
    return(pd.Series([slope,intercept,rvalu,pvalue,stderr,len(xf)],\
                     index=['slope','intercept','rvalu','pvalue','stderr','count']))

regFuelSalesVsAppPriceADO_pct = data.groupby('Site Name').apply(regRes).reset_index()

regFuelSalesVsAppPriceADO_pct_sig = regFuelSalesVsAppPriceADO_pct.query('pvalue <= 0.05')

In [67]:
## elasticity wrt UL95
def regRes_gen(site,x,y):
    # filter all data of the site between 5th & 95th precentile 
    q_5,q_95 = site[y].quantile([0.02,0.98]).values.round().tolist()
    xf = site.loc[(site[y] >= q_5)& (site[y] <= q_95)]
    slope,intercept,rvalu,pvalue,stderr= stats.linregress(xf[x],xf[y])
    return(pd.Series([slope,intercept,rvalu,pvalue,stderr,len(xf)],\
                     index=['slope','intercept','rvalu','pvalue','stderr','count']))

# exclusive shop sales 
regExclShopSalesVsAppPriceUL95_pct = data.groupby('Site Name')\
                    .apply(lambda site: regRes_gen(site,'UL95AppPriceChgPct',\
                                                        'pctChgExclShopSalesFromMedian')).reset_index()
regExclShopSalesVsAppPriceUL95_pct_sig = regExclShopSalesVsAppPriceUL95_pct.query('pvalue <= 0.05')


# shop sales from fuel sales
regShopSalesFromFuelVsAppPriceUL95_pct = data.groupby('Site Name')\
                    .apply(lambda site: regRes_gen(site,'UL95AppPriceChgPct',\
                                                        'pctChgShopSalesFromFuelFromMedian')).reset_index()
regShopSalesFromFuelVsAppPriceUL95_pct_sig = regShopSalesFromFuelVsAppPriceUL95_pct.query('pvalue <= 0.05')


# fuel sales
regFuelSalesVsAppPriceUL95_pct = data.groupby('Site Name')\
                    .apply(lambda site: regRes_gen(site,'UL95AppPriceChgPct',\
                                                        'pctChgFuelSalesFromMedian')).reset_index()
regFuelSalesVsAppPriceUL95_pct_sig = regFuelSalesVsAppPriceUL95_pct.query('pvalue <= 0.05')

In [78]:
regShopSalesFromFuelVsAppPriceADO_pct_sig.mean()

slope       -0.21
intercept    1.17
rvalu       -0.03
pvalue       0.01
stderr       1.08
dtype: float64

In [84]:
reqColumns = ['Site Number', 'Site Name', 'Site ID', 'PBL','Transaction Date', 'Total Sales Value',\
               'pctChgExclShopSalesFromMedian','pctChgShopSalesFromFuelFromMedian',\
                'pctChgFuelSalesFromMedian','AdoApprPriceChgPct', 'UL95AppPriceChgPct',\
               'UL 95_ Approved Price Vs Marker Price','UL 95_ Approved Price Vs Marker Price',\
               'Total Sales Value','exclShopSales','shopSalesFromFuel','totalSalesEsso']
data.loc[:,reqColumns].to_excel('data_tableau_elasticity.xlsx',index=False)

In [87]:
regFuelSalesVsAppPriceADO_pct_sig.sort_values('slope',ascending=True)
regFuelSalesVsAppPriceADO_pct_sig.sort_values('slope',ascending=False)

Site Name  slope  intercept  rvalu  pvalue  stderr
156         SLOUGH EXPRESS -10.66       8.81  -0.71    0.00    2.40
186      WIMBLEDON EXPRESS  -9.07      -3.60  -0.41    0.00    1.16
51         CLIFTON LEE EXP  -8.14      11.76  -0.64    0.00    0.55
84     FOREST HILL EXPRESS  -5.79       3.34  -0.37    0.00    0.82
49      CHURCHTOWN EXPRESS  -5.60      -0.26  -0.52    0.00    0.54
102  HESKETH SOUTHPORT EXP  -5.55       2.06  -0.40    0.00    0.73
52   COLLIERS WOOD EXPRESS  -5.38       0.15  -0.37    0.00    0.77
17      BILLY MILL EXPRESS  -4.75       2.60  -0.36    0.00    0.69
14        BEARSTED EXPRESS  -4.75       0.38  -0.43    0.00    0.57
70   EASTWOOD RAYLG RD EXP  -4.44       3.18  -0.30    0.00    0.84
23   BRAMHALL CHESHIRE EXP  -4.30      -3.88  -0.20    0.00    1.25
120   LOAMPIT VALE EXPRESS  -3.99      -3.38  -0.34    0.00    0.64
95          HARROW EXPRESS  -3.97      -2.14  -0.29    0.00    0.81
55          COPNOR EXPRESS  -3.90       3.90  -0.26    0.00    0.83
43      CHAPELTOWN EXPRESS  -3.81      -2.43  -0.29    0.00    0.73
130    NORTHWOOD HILLS EXP  -3.80       1.99  -0.23    0.00    0.89
194        YATELEY EXPRESS  -3.78      -1.37  -0.33    0.00    0.60
193  WYCOMBE SOUTH EXPRESS  -3.73       2.91  -0.40    0.00    0.50
142      PORTSWOOD EXPRESS  -3.60       5.79  -0.21    0.00    0.94
190    WOODINGDEAN EXPRESS  -3.59       6.41  -0.38    0.00    0.50
72       EDINBURGH EXPRESS  -3.52       4.42  -0.25    0.00    0.78
16   BEWDLEY HILL KIDD EXP  -3.51       5.13  -0.36    0.00    0.51
137   OULTON BROAD EXPRESS  -3.44       5.46  -0.34    0.00    0.53
180  WEYMOUTH LANEH RD EXP  -3.44       2.88  -0.17    0.00    1.16
10         BARKING EXPRESS  -3.32       3.04  -0.23    0.00    0.84
151       RUSHOLME EXPRESS  -3.31       3.41  -0.26    0.00    0.67
4       ADDISCOMBE EXPRESS  -3.29       2.37  -0.22    0.00    0.84
45         CHELSEA EXPRESS  -3.26       2.66  -0.23    0.00    0.79
2    ACKLAM RD MIDSBRO EXP  -3.22       3.52  -0.37    0.00    0.45
111      KIRK ELLA EXPRESS  -3.17      -2.99  -0.32    0.00    0.51
..                     ...    ...        ...    ...     ...     ...
110   KINGSWINFORD EXPRESS  -1.27      -2.10  -0.22    0.00    0.33
131  NOTTM RD M MOWBRY EXP  -1.26      -2.72  -0.14    0.02    0.56
65   DOSTHILL TAMWORTH EXP  -1.24       2.64  -0.16    0.00    0.44
119         LEYTON EXPRESS  -1.23       1.28  -0.16    0.00    0.43
192  WYCOMBE NORTH EXPRESS  -1.23       1.28  -0.12    0.03    0.55
66    DUKERIES MANSFLD EXP  -1.22      -1.25  -0.13    0.03    0.55
128  NEWPORT CHPSTW RD EXP  -1.22       0.21  -0.12    0.04    0.59
29   BULLSMRE LN WAL X EXP  -1.18      -0.04  -0.12    0.04    0.57
101      HERMITAGE EXPRESS  -1.10       1.42  -0.24    0.00    0.27
118   LEIGH ON SEA EXPRESS  -1.09      -0.39  -0.23    0.00    0.27
71   ECCLES LIVERPL RD EXP  -1.02      -0.77  -0.16    0.00    0.35
30            BURY EXPRESS  -0.82      -0.60  -0.14    0.01    0.33
133    OLDSWINFORD EXPRESS  -0.66       1.54  -0.12    0.03    0.31
146  RIDGECLOSE KETTER EXP   0.39       0.57   0.11    0.05    0.19
87   FROGWELL CHIPPNHM EXP   0.71      -0.74   0.12    0.04    0.35
15   BESSACARR DONCAST EXP   0.84      -1.09   0.20    0.00    0.24
163       STAFFORD EXPRESS   0.96       0.63   0.18    0.00    0.29
139         PENDLEBURY EXP   1.10      -0.61   0.14    0.02    0.46
90        GOWERTON EXPRESS   1.40      -0.39   0.15    0.01    0.52
24         BRECHIN EXPRESS   1.64      -5.21   0.24    0.00    0.38
159    SOUTH OXHEY EXPRESS   1.88      -0.39   0.16    0.00    0.63
92      GROVE PARK EXPRESS   2.22       0.47   0.19    0.00    0.66
155  SILKSWRTH RD SUND EXP   2.26      -1.53   0.23    0.00    0.60
50        CLIFTON LANE EXP   2.32      -2.09   0.22    0.00    0.71
32    BUSHEY HEATH EXPRESS   2.35      -5.17   0.17    0.00    0.78
82           FLEET EXPRESS   2.36      -3.80   0.15    0.02    1.04
18        BITTERNE EXPRESS   2.56      -3.01   0.16    0.00    0.89
167  

In [110]:
site_id_name = data.loc[:,['PBL', 'Site Name']].drop_duplicates()
regFuelSalesVsAppPriceADO_pct = regFuelSalesVsAppPriceADO_pct.merge(site_id_name,how = 'left',on='Site Name')
regFuelSalesVsAppPriceADO_pct.to_excel(r'fuelElasticity_regression.xlsx')
regFuelSalesVsAppPrice = regFuelSalesVsAppPriceADO_pct.merge(regFuelSalesVsAppPriceUL95_pct,how = 'left',on='Site Name')
PDI_elasticity_results = pd.read_excel(r'PDI_elasticity_results.xlsx')
regFuelSalesVsAppPrice_pdi = regFuelSalesVsAppPrice.merge(PDI_elasticity_results,\
                        how = 'left',left_on='PBL',right_on='Site Id')
selCols = ['Site Name', 'slope_x','EL ADO','Elasticity ADO','rvalu_x', 'pvalue_x',\
           'slope_y', 'EL UL' ,'Elasticity UL','EL TOTAL',\
          'rvalu_y','pvalue_y','Site Id', 'Site','Site Number', 'PBL']
df_el = regFuelSalesVsAppPrice_pdi.loc[:,selCols]
df_el.to_excel('elasticity_calcuation_PDI_vs_regression.xlsx',index=False)

In [127]:
data.to_pickle('df_tesco_marker_V3.pkl')

df_el.to_pickle('df_el.pkl')

In [131]:
regShopSalesFromFuelVsAppPriceADO_pct['label']= 'shopSalesFromFuelADO'
regShopSalesFromFuelVsAppPriceUL95_pct['label']= 'shopSalesFromFuelUL95'
regExclShopSalesVsAppPriceADO_pct['label'] = 'exclShopSalesADO'
regExclShopSalesVsAppPriceUL95_pct['label'] = 'exclShopSalesUL95'

In [132]:
shpSales_el = pd.concat([regShopSalesFromFuelVsAppPriceADO_pct,regShopSalesFromFuelVsAppPriceUL95_pct,\
          regExclShopSalesVsAppPriceADO_pct,regExclShopSalesVsAppPriceUL95_pct],axis=0)

In [135]:
shpSales_el.to_excel(r'elasticity_shopSales_regression.xlsx',index=False)

In [146]:
## elasticity wrt UL95
def regRes_gen_1(site,x,y):
    # filter all data of the site between 5th & 95th precentile 
    q_5,q_95 = site[y].quantile([0.02,0.98]).values.round().tolist()
    xf = site.loc[(site[y] >= q_5)& (site[y] <= q_95)]
    xf[y] = xf[y] - xf[y].mean()
    slope,intercept,rvalu,pvalue,stderr= stats.linregress(xf[x],xf[y])
    return(pd.Series([slope,intercept,rvalu,pvalue,stderr,len(xf)],\
                     index=['slope','intercept','rvalu','pvalue','stderr','count']))

In [147]:
regFuelSalesVsAppPriceADO_pct = data.groupby('Site Name')\
                    .apply(lambda site: regRes_gen_1(site,'AdoApprPriceChgPct',\
                                                        'pctChgFuelSalesFromMedian')).reset_index()

In [ ]:
regFuelSalesVsAppPriceADO_pct.head()

Site Name  slope  intercept  rvalu  pvalue  stderr  count
0  ABBEY ST ACCRNGTN EXP  -1.89       1.58  -0.14    0.01    0.74 319.00
1       ABINGTON EXPRESS  -2.04       0.55  -0.13    0.02    0.88 308.00
2  ACKLAM RD MIDSBRO EXP  -3.22       1.80  -0.37    0.00    0.45 318.00
3          ACTON EXPRESS   0.63      -0.34   0.05    0.34    0.66 305.00
4     ADDISCOMBE EXPRESS  -3.29       3.07  -0.22    0.00    0.84 316.00

In [142]:
regFuelSalesVsAppPriceADO_pct.head()

Site Name  slope  intercept  rvalu  pvalue  stderr  count
0  ABBEY ST ACCRNGTN EXP  -1.89       0.20  -0.14    0.01    0.74 319.00
1       ABINGTON EXPRESS  -2.04       0.84  -0.13    0.02    0.88 308.00
2  ACKLAM RD MIDSBRO EXP  -3.22       3.52  -0.37    0.00    0.45 318.00
3          ACTON EXPRESS   0.63      -1.44   0.05    0.34    0.66 305.00
4     ADDISCOMBE EXPRESS  -3.29       2.37  -0.22    0.00    0.84 316.00